# Busienss Transformation and Modeling
 

In [0]:
query = """
SELECT 
    ROW_NUMBER() OVER (ORDER BY ci.customer_id) AS customer_key,
    ci.customer_id,
    ci.customer_number,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE 
        WHEN ci.gender != 'N/A' THEN ci.gender
        ELSE COALESCE(ca.gender, 'N/A')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.created_date AS create_date
FROM silver.crm_customers ci
LEFT JOIN silver.erp_cust_az12 ca
    ON ci.customer_number = ca.customer_number
LEFT JOIN silver.erp_loc_a101 la
    ON ci.customer_number = la.customer_number
"""
df = spark.sql(query)

In [0]:
df.limit(10).display()

# Write to Gold Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("gold.dim_customers")

In [0]:
%sql
SELECT * FROM workspace.gold.dim_customers LIMIT 10